<a href="https://colab.research.google.com/github/shubhamgoel90/Data-Modeling/blob/master/Crop_AI_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files

In [0]:
# Install Kaggle library
!pip install -q kaggle

In [0]:
# Upload kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!kaggle datasets download -d emmarex/plantdisease

100% 657M/658M [00:12<00:00, 54.6MB/s]
100% 658M/658M [00:12<00:00, 53.2MB/s]


In [0]:
%%capture
!unzip plantdisease.zip

In [0]:
!ls

kaggle.json  plantdisease.zip  plantvillage  PlantVillage  sample_data


In [0]:
!ls PlantVillage

Pepper__bell___Bacterial_spot  Tomato_Late_blight
Pepper__bell___healthy	       Tomato_Leaf_Mold
Potato___Early_blight	       Tomato_Septoria_leaf_spot
Potato___healthy	       Tomato_Spider_mites_Two_spotted_spider_mite
Potato___Late_blight	       Tomato__Target_Spot
Tomato_Bacterial_spot	       Tomato__Tomato_mosaic_virus
Tomato_Early_blight	       Tomato__Tomato_YellowLeaf__Curl_Virus
Tomato_healthy


In [0]:
!ls plantvillage

PlantVillage


In [0]:
import os
folders = os.listdir("PlantVillage")
print(folders)

['Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Leaf_Mold', 'Potato___healthy', 'Tomato__Tomato_mosaic_virus', 'Pepper__bell___Bacterial_spot', 'Tomato_Bacterial_spot', 'Potato___Late_blight', 'Tomato_Early_blight', 'Pepper__bell___healthy', 'Tomato__Target_Spot', 'Potato___Early_blight', 'Tomato_Late_blight', 'Tomato_healthy', 'Tomato_Septoria_leaf_spot']


In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale = 1.0/255)

train_generator = train_gen.flow_from_directory("PlantVillage", target_size = (224,224), batch_size = 64, class_mode = "categorical", shuffle = True)


Using TensorFlow backend.


Found 20638 images belonging to 15 classes.


# New Section

In [0]:
train_generator.labels

array([ 0,  0,  0, ..., 14, 14, 14], dtype=int32)

In [0]:
x, y = train_generator.next()
print(x.shape)
print(y.shape)

(64, 224, 224, 3)
(64, 15)


In [0]:
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model


In [0]:
model = ResNet50(include_top =False, weights = "imagenet", input_shape = (224,224,3))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 7s 0us/step


In [0]:
av1 = GlobalAveragePooling2D()(model.output)
fc1 = Dense(256 , activation = "relu")(av1)
d1 = Dropout(0.4)(fc1)
fc2 = Dense(15 , activation= "softmax")(d1)

model_new = Model(input = model.input, output = fc2)
model_new.summary()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [0]:
adam = Adam(lr = 0.00003)
model_new.compile(loss = "categorical_crossentropy", optimizer ="adam", metrics = ["accuracy"] )

In [0]:
for ix in range(169):
    model_new.layers[ix].trainable = False
model_new.compile(loss = "categorical_crossentropy", optimizer ="adam", metrics = ["accuracy"] )
model_new.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [24]:
hist = model_new.fit_generator(train_generator, epochs = 5, steps_per_epoch= 325)

model_new.save("CropAI_model_train_gen_epoch_5.h5")
print("model weights are save on disk")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
325/325 [==============================] - 50s 154ms/step - loss: 0.4260 - acc: 0.8653
Epoch 2/5
325/325 [==============================] - 42s 130ms/step - loss: 0.1368 - acc: 0.9554
Epoch 3/5
325/325 [==============================] - 42s 130ms/step - loss: 0.0803 - acc: 0.9737
Epoch 4/5
325/325 [==============================] - 42s 128ms/step - loss: 0.0586 - acc: 0.9801
Epoch 5/5
325/325 [==============================] - 42s 129ms/step - loss: 0.0412 - acc: 0.9861
model weights are save on disk
